# Species identification

This notebook allows the user to display small windows around the emission lines of a molecular species to confirm the presence of the species in a spectrum data file. An optional telescope information file can be selected to convert antenna temperature to main beam temperature. This has been tested with Yebes 40m data, other telescopes may need additional code adjustments to correctly read telescope information.

* Press q to go to the next line

This notebook uses tables taken from splatalogue in .tsv format. Input variables can be modified to select different parts of the spectrum and range in MHz of the window.

User input:

In [ ]:
# Data files
spectrum_file = "/home/david/Documents/MasterAstro/practicas-master/Data/G10-47_all_reduced.fits"
telescope_info_file = "/home/david/Documents/MasterAstro/practicas-master/Data/Yebes_40m_W_band.csv"
molecule_file = "/home/david/Documents/MasterAstro/practicas-master/Data/Molecule_lines/H2NCH2CN.tsv"
# Use main beam temperature or antenna temperature
use_TMB = True
# Input parameters
zoom_range = (-15, 20)  # in MHz

Import libraries.

In [ ]:
import astropy.constants as const
from astropy.io import fits
from astropy.nddata import CCDData
from astropy.stats import sigma_clipped_stats
from astropy.table import Table, QTable
import astropy.units as u
from astropy.wcs import WCS
import matplotlib as mpl
import matplotlib.axes as maxes
import matplotlib.patches as patches
import matplotlib.patheffects as PathEffects
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import numpy.ma as ma
from pathlib import Path
import scipy.interpolate

from prac_utils import find_nearest_idx
from prac_utils import x2freq

Read the spectrum data file and beam efficiency from telescope info.

In [ ]:
# Read full spectra
ccd = CCDData.read(spectrum_file)

# Read telescope data obtained from its webpage
if use_TMB:
    telescope_info = QTable.read(telescope_info_file, format='ascii')
    telescope_info['freq'] = telescope_info['freq'] * 1e3 * u.MHz
    telescope_info['lambda'].unit = u.mm
    telescope_info['theta'].unit = u.arcsec
    telescope_info['Jy/K'].unit = u.Jy / u.K

Create interpolation data of the telescope.

In [ ]:
# Create interpolation and extrapolation for the beam efficiency

if use_TMB:
    pol = 'V'
    x = telescope_info['freq'][telescope_info['polarization']==pol]
    y = telescope_info['eta_MB'][telescope_info['polarization']==pol]
    interp_band_W = scipy.interpolate.interp1d(x, y, fill_value='extrapolate')

Use tkinter backend for matplotlib

In [ ]:
# Use this instead of "%matplotlib tk" to show a window only after the previous one is closed
mpl.use('TkAgg')

Read molecule data from table

In [ ]:
input_lines = Table.read(molecule_file, format='ascii', delimiter='\t')
# this creates one single vector from data from the two freq columns
first = input_lines['Freq-MHz(rest frame,redshifted)']
second = input_lines['Meas Freq-MHz(rest frame,redshifted)']
molecule_line_freqs = np.zeros(len(first))
for i in range(0, len(first)):
    if ma.is_masked(first[i]):
        molecule_line_freqs[i] = second[i]
    else:
        molecule_line_freqs[i] = first[i]
molecule_line_QNs = input_lines['Resolved QNs']
molecule_line_E_Ls = input_lines['E_L (K)']
molecule_line_intensities = input_lines['CDMS/JPL Intensity']
molecule_line_catalogues = input_lines['Linelist']

Show line information.

In [ ]:
y_axis = ccd.data[0, 0, 0, :]    # need to check slicing in header
freq_axis = x2freq(ccd=ccd) * 1e-6    # convert to MHz
if ccd.header['BUNIT'] == 'K':
    if use_TMB:
        etas = interp_band_W(freq_axis)
        T_beam = y_axis / etas
        y = T_beam
    else:
        y = y_axis
else:
    y = y_axis
freq_axis = x2freq(ccd=ccd) * 1e-6    # convert to MHz

# Main loop
for i in range(len(molecule_line_freqs)):

    fig, ax, = plt.subplots()
    ax.plot(freq_axis, y)

    # set figure x limits
    xlim = (molecule_line_freqs[i]+zoom_range[0], molecule_line_freqs[i]+zoom_range[1])
    ax.set_xlim(xlim)
    # set figure y limits
    center_idx = find_nearest_idx(freq_axis, molecule_line_freqs[i])    # paso valores de freq a Hz
    min_idx = find_nearest_idx(freq_axis, xlim[0])
    max_idx = find_nearest_idx(freq_axis, xlim[1])
    min_y = min(y[min_idx:max_idx])
    max_y = max(y[min_idx:max_idx])
    ylim = (min_y - 0.15*(max_y-min_y), max_y + 0.2*(max_y-min_y))
    ax.set_ylim(ylim)

    # figure design
    ax.set_xlabel('Freq (MHz)')
    if ccd.header['BUNIT'] == 'K':
        if use_TMB:
            ax.set_ylabel(r'$T_{\rm MB}$ (K)')
        else:
            ax.set_ylabel(r'$T_{\rm A}^{*}$ (K)')
    else:
        ax.set_ylabel('Y axis')

    # print text in screen about the line
    ax.axvline(molecule_line_freqs[i], color='C3')
    separation = 0.03*(ax.get_xlim()[1]-ax.get_xlim()[0])
    ax.text(molecule_line_freqs[i]-separation, ylim[0]+0.96*(ylim[1]-ylim[0]), f'Catalogue: {molecule_line_catalogues[i]}', transform=ax.transData, fontsize=9,
                    color='black', rotation='horizontal', horizontalalignment='right', verticalalignment='center')
    ax.text(molecule_line_freqs[i]+separation, ylim[0]+0.96*(ylim[1]-ylim[0]), f'QNs: {molecule_line_QNs[i]}', transform=ax.transData, fontsize=9,
                    color='C1', rotation='horizontal', horizontalalignment='left', verticalalignment='center')
    ax.text(molecule_line_freqs[i]+separation, ylim[0]+0.91*(ylim[1]-ylim[0]), r'$E_{\rm low}$ = %s K' %(round(molecule_line_E_Ls[i], 3)), transform=ax.transData, fontsize=9,
                    color='C2', rotation='horizontal', horizontalalignment='left', verticalalignment='center')
    ax.text(molecule_line_freqs[i]-separation, ylim[0]+0.91*(ylim[1]-ylim[0]), r'$I$ = %s' %(round(molecule_line_intensities[i], 3)), transform=ax.transData, fontsize=9,
                    color='C3', rotation='horizontal', horizontalalignment='right', verticalalignment='center')
    plt.show()